In [1]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import numpy as np

### Data Loading 

In [4]:
df = pd.read_csv('Iowa_Liquor_Sales_DEC2023.csv')

/var/folders/m7/s2pnwq9j1tj4h07p5sm0s1tw0000gn/T/ipykernel_43537/3037590991.py:1: DtypeWarning: Columns (6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Iowa_Liquor_Sales_DEC2023.csv')


In [5]:
pd.set_option('display.max_columns', None)
df.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S03394400014,01/03/2012,4334,FAREWAY STORES #114 / DUBUQUE,2050 JOHN F KENNEDY RD,DUBUQUE,52001,NaN,31.0,DUBUQUE,1011100.0,BLENDED WHISKIES,260.0,DIAGEO AMERICAS,25601,SEAGRAMS 7 CROWN BL WHISKEY MINI,12,500,6.17,9.25,2,18.50,1.0,0.26
1,S03381400069,01/03/2012,3354,SAM'S CLUB 8238 / DAVENPORT,3845 ELMORE AVE.,DAVENPORT,52807,POINT (-90.527081 41.559731),82.0,SCOTT,1082900.0,MISC. IMPORTED CORDIALS & LIQUEURS,330.0,GEMINI SPIRITS,66936,GRANGALA TRIPLE ORANGE LIQUEUR,12,750,10.99,16.49,36,593.64,27.0,7.13
2,S03384600026,01/03/2012,2190,"CENTRAL CITY LIQUOR, INC.",1460 2ND AVE,DES MOINES,50314,POINT (-93.619787 41.60566),77.0,POLK,1012100.0,CANADIAN WHISKIES,260.0,DIAGEO AMERICAS,11297,CROWN ROYAL CANADIAN WHISKY,12,1000,17.46,26.19,24,628.56,24.0,6.34
3,S03391400005,01/03/2012,4272,FAREWAY STORES #407 / ESTERVILLE,1108 CENTRAL AVE,ESTHERVILLE,51334,POINT (-94.829963 43.402096),32.0,EMMET,1012200.0,SCOTCH WHISKIES,55.0,SAZERAC NORTH AMERICA,8828,LAUDER'S,6,1750,11.18,16.76,6,100.56,10.5,2.77
4,S03410300009,01/03/2012,3495,GREAT PASTIMES,228 N MAIN ST,MONTICELLO,52310,POINT (-91.183413 42.24013200000001),53.0,JONES,1012100.0,CANADIAN WHISKIES,115.0,"CONSTELLATION WINE COMPANY, INC.",11786,BLACK VELVET TRAVELER,12,750,4.83,7.44,12,89.28,9.0,2.38


### Clean Store Location to Lat and Lon

In [7]:
import re

def extract_lat_lon(point):
    if isinstance(point, str):  # Check if the input is a string
        nums = re.findall(r'[-]?\d+\.\d+', point)
        if len(nums) == 2:  # Ensure that two numbers are found
            return float(nums[1]), float(nums[0])  # Latitude is the second number, Longitude is the first
    return float('nan'), float('nan')  # Return NaN if input is not a valid string

# Apply the function to the 'Store Location' column
df['Lat'], df['Lon'] = zip(*df['Store Location'].apply(extract_lat_lon))

# Print the updated DataFrame
print(df[['Store Location', 'Lat', 'Lon']])

                                Store Location        Lat        Lon
0                                          NaN        NaN        NaN
1                 POINT (-90.527081 41.559731)  41.559731 -90.527081
2                  POINT (-93.619787 41.60566)  41.605660 -93.619787
3                 POINT (-94.829963 43.402096)  43.402096 -94.829963
4         POINT (-91.183413 42.24013200000001)  42.240132 -91.183413
...                                        ...        ...        ...
27934716    POINT (-91.633605478 41.976639164)  41.976639 -91.633605
27934717    POINT (-93.613286034 41.605834999)  41.605835 -93.613286
27934718     POINT (-92.440126032 42.48685198)  42.486852 -92.440126
27934719     POINT (-92.327689003 42.45826902)  42.458269 -92.327689
27934720    POINT (-93.262495004 43.145527999)  43.145528 -93.262495

[27934721 rows x 3 columns]


In [8]:
df.drop(['Store Location'], axis=1, inplace=True)

In [9]:
# for Nan value in Lat and Lon, fill with -1

df['Lat'] = df['Lat'].fillna(-1)
df['Lon'] = df['Lon'].fillna(-1)

### To Datetime

In [11]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

In [12]:
# rename df["Sale (Dollars)"] to Revenue

df.rename(columns={'Sale (Dollars)': 'Revenue'}, inplace=True)

### Check for duplicate items

In [14]:
# check if we have any duplicates in the data

dupe_all = df.duplicated().sum()
dupe_all

0

### Clean vendor name

In [16]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Drop rows with any missing 'Vendor Number' or 'Vendor Name'
clean_df = df.dropna(subset=['Vendor Number', 'Vendor Name'])

# Group by 'Vendor Number' and collect all unique 'Vendor Name' entries per 'Vendor Number'
vendor_groups = clean_df.groupby('Vendor Number')['Vendor Name'].agg(lambda x: set(x.unique()))

# Filter groups where the number of unique 'Vendor Name' entries is more than one
multiple_vendors = vendor_groups[vendor_groups.apply(lambda x: len(x) > 1)]

# Create a DataFrame for display
multiple_vendors_df = multiple_vendors.reset_index()

# Rename the columns for better readability
multiple_vendors_df.columns = ['Vendor Number', 'Vendor Names']

multiple_vendors_df

,Vendor Number,Vendor Names
0,10.0,"{A HARDY / U.S.A., LTD., A HARDY USA LTD}"
1,35.0,"{BACARDI U.S.A., INC., BACARDI USA INC}"
2,79.0,"{BRECKENRIDGE DISTILLERY / DOUBLE DIAMOND DISTILLERY LLC, BRECKENRIDGE DISTILLERY}"
3,85.0,"{BROWN FORMAN CORP., BROWN-FORMAN CORPORATION}"
4,86.0,"{BROADBENT VENTURES LLC, BROADBENT DISTILLERY}"
5,90.0,"{CARRIAGE HOUSE IMPORTS, LTD., CARRIAGE HOUSE IMPORTS}"
6,107.0,"{BLAUM BROS. DISTILLING CO., BLAUM BROTHERS DISTILLING CO}"
7,108.0,"{CVI BRANDS, CVI BRANDS / CALIFORNIA VINEYARDS INC}"
8,110.0,"{3 BADGE MIXOLOGY, 3 BADGE CORPORATION}"
9,114.0,"{AHA TORO SPIRITS INC, AHA TORO SPIRITS INC / AHA YETO TEQUILA}"


#### Normalize vendor names (duplicate and misspell)

In [18]:
standardized_names = {
    10.0: 'A HARDY USA LTD',
    35.0: 'BACARDI USA INC',
    79.0: 'BRECKENRIDGE DISTILLERY',
    85.0: 'BROWN FORMAN CORP.',
    86.0: 'BROADBENT DISTILLERY',
    90.0: 'CARRIAGE HOUSE IMPORTS',
    107.0: 'BLAUM BROTHERS DISTILLING CO',
    108.0: 'CVI BRANDS',
    110.0: '3 BADGE CORPORATION',
    114.0: 'AHA TORO SPIRITS INC',
    115.0: 'CONSTELLATION BRANDS INC',
    116.0: 'BAD BEAR ENTERPRISES LLC',
    121.0: 'COOPER SPIRITS INTERNATIONAL',
    125.0: 'CEDAR RIDGE VINEYARDS LLC',
    130.0: 'DISARONNO INTERNATIONAL LLC',
    136.0: 'HIGH WEST DISTILLERY',
    150.0: 'DREYFUS ASHBY',
    154.0: 'IOWA DISTILLING COMPANY',
    156.0: 'DOMAINE SELECT WINE ESTATES',
    163.0: 'MARSALLE COMPANY',
    192.0: 'SIDNEY FRANK IMPORTING CO.',
    198.0: 'FRANK-LIN DISTILLERS PRODUCTS LTD.',
    204.0: 'HILLROCK ESTATE DISTILLERY',
    205.0: 'E & J GALLO WINERY',
    214.0: 'WHISTLEPIG LLC',
    224.0: 'STE MICHELLE WINE ESTATES',
    226.0: 'MADIKWE USA, INC.',
    239.0: 'RUSSIAN STANDARD VODKA',
    240.0: 'WILLIAM GRANT & SONS INC',
    244.0: 'FILIBUSTER BARRELS LLC',
    255.0: 'WILSON DANIELS LTD.',
    258.0: 'PURPLE VALLEY IMPORTS',
    267.0: 'IMPERIAL BRANDS INC',
    268.0: 'IMPEX BEVERAGE INC',
    279.0: 'TOPA SPIRITS, LLC',
    285.0: 'CHARLES JACQUIN ET CIE',
    287.0: 'CHATHAM IMPORTS INC',
    288.0: 'KINDRED SPIRITS OF NORTH AMERICA',
    293.0: 'JINRO AMERICA, INC',
    294.0: 'LEVECKE CORPORATION',
    295.0: 'KOBRAND CORP',
    297.0: 'LAIRD AND COMPANY',
    300.0: 'MCCORMICK DISTILLING CO.',
    301.0: 'FIFTH GENERATION INC',
    306.0: 'MISSISSIPPI RIVER DISTILLING CO.',
    308.0: 'MANGO BOTTLING',
    310.0: 'STOLLER WAREHOUSE',
    311.0: 'MODERN SPIRIT LLC',
    312.0: 'KLIN SPIRITS, LLC',
    313.0: 'VISION WINE & SPIRIT LLC',
    315.0: 'PATRIARCH DISTILLERS',
    322.0: 'PRESTIGE WINE & SPIRITS GROUP',
    334.0: 'DES MOINES IMPORTS',
    336.0: 'MAVERICK SPIRIT LLC',
    338.0: 'NICHE IMPORT CO.',
    343.0: 'PACIFIC EDGE WINE AND SPIRITS',
    346.0: 'OLE SMOKY DISTILLERY, LLC',
    347.0: 'NORTH SHORE DISTILLERY',
    357.0: 'PALM BAY INTERNATIONAL',
    363.0: 'PATERNO IMPORTS, LTD.',
    370.0: 'PERNOD RICARD USA',
    380.0: 'PHILLIPS BEVERAGE COMPANY',
    384.0: 'PIEDMONT DISTILLERS INC',
    389.0: 'REMY COINTREAU USA INC',
    391.0: 'ANCHOR DISTILLING',
    399.0: 'ROGUE ALES AND SPIRITS',
    410.0: 'PATRON SPIRITS COMPANY',
    418.0: 'GUNSANDMORE.INFO',
    420.0: 'MOET HENNESSY USA, INC.',
    421.0: 'SAZERAC CO., INC.',
    428.0: 'TRAVERSE CITY WHISKEY CO',
    434.0: 'LUXCO INC',
    448.0: 'ST. GEORGE SPIRIT, INC',
    452.0: 'SPEAKEASY SPIRITS',
    456.0: 'SPIRIT IMPORTS INC',
    459.0: 'SURVILLE ENTERPRISES CORP',
    460.0: 'SHAW-ROSS INTERNATIONAL',
    461.0: 'SKYY SPIRITS INC',
    469.0: 'TY KU, LLC',
    477.0: 'TOTAL BEVERAGE SOLUTIONS',
    478.0: 'TRINCHERO FAMILY ESTATES',
    479.0: 'VIN DIVINO LTD',
    482.0: 'SOVEREIGN BRANDS',
    492.0: 'WESTERN SPIRITS BEVERAGE CO. LLC',
    495.0: 'MS WALKER INC',
    497.0: 'FREDRICK WILDMAN & SON',
    501.0: 'WINEBOW INC',
    511.0: 'KINGS COUNTY DISTILLERY',
    521.0: 'JEM BEVERAGE COMPANY',
    578.0: 'S&B FARMS DISTILLERY',
    594.0: 'AMERICAN HERITAGE DISTILLERS, LLC',
    620.0: 'MPL BRANDS NV INC',
    626.0: 'RED BOOT DISTILLERY',
    641.0: 'CH DISTILLERY',
    785.0: 'MAISON FERRAND USA',
    803.0: 'LEVECKE CORPORATION',
    885.0: 'YAHARA BAY DISTILLERS, INC',
    886.0: 'GEORGETOWN TRADING CO. LLC',
    962.0: 'DUGGANS DISTILLERS',
    969.0: 'AVENIU BRANDS',
    971.0: 'HOOD RIVER DISTILLERS',
    977.0: 'WERNER DISTILLING',
    978.0: 'W J DEUTSCH & SONS LTD',
}

In [ ]:
# Function to standardize vendor names
def standardize_names(vendor_id, current_name):
    return standardized_names.get(vendor_id, current_name)

# Apply the standardization
df['Vendor Name'] = df.apply(lambda row: standardize_names(row['Vendor Number'], row['Vendor Name']), axis=1)

#### Normalize category name

In [ ]:
df["Category Name"].unique()

In [ ]:
# some duplicate values in the category name

df['Category Name'] = df['Category Name'].str.replace('SCHNAPPS - IMPORTED', 'IMPORTED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('AMARETTO - IMPORTED', 'IMPORTED AMARETTO')
df['Category Name'] = df['Category Name'].str.replace('IMPORTED VODKA - MISC', 'IMPORTED VODKAS')
df['Category Name'] = df['Category Name'].str.replace('MISC. AMERICAN CORDIALS & LIQUEURS', 'AMERICAN CORDIALS & LIQUEURS')
df['Category Name'] = df['Category Name'].str.replace('MISC. IMPORTED CORDIALS & LIQUEURS', 'IMPORTED CORDIALS & LIQUEURS')
df['Category Name'] = df['Category Name'].str.replace('FLAVORED GINS', 'FLAVORED GIN')
df['Category Name'] = df['Category Name'].str.replace('VODKA FLAVORED', 'FLAVORED VODKAS')
df['Category Name'] = df['Category Name'].str.replace('WHISKEY LIQUEURS', 'WHISKEY LIQUEUR')
df['Category Name'] = df['Category Name'].str.replace('IMPORTED GRAPE BRANDIES', 'IMPORTED BRANDIES')
df['Category Name'] = df['Category Name'].str.replace('ROOT BEER SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('PEACH SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('APPLE SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('WATERMELON SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('STRAWBERRY SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('RASPBERRY SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('PEPPERMINT SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('MISCELLANEOUS SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('GRAPE SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('CHERRY SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('BUTTERSCOTCH SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('CINNAMON SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('PEPPERMINT SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('TROPICAL FRUIT SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('SPEARMINT SCHNAPPS', 'FLAVORED SCHNAPPS')
df['Category Name'] = df['Category Name'].str.replace('CREAME LIQUEURS', 'CREAM LIQUEURS')
df['Category Name'] = df['Category Name'].str.replace('COCKTAILS /RTD', 'COCKTAILS/RTD')

#### Check null values in category name based on item description

In [ ]:
# count the percentage of the null values in the 'Category Name' column

unique_categories = df['Category Name'].nunique()

null_categories = df['Category Name'].isnull().sum()

(unique_categories / null_categories)*100

#### Drop null category name

In [ ]:
# Drop the rest of the null values in 'Category Name'

df.dropna(subset=['Category Name'], inplace=True)

### Adding new column - Main Category

In [ ]:
def map_category(category_name):
    # Define mappings for various types
    if 'WHISK' in category_name or 'BOURBON' in category_name or 'SCOTCH' in category_name or 'RYE' in category_name:
        return 'WHISKY'
    elif 'GIN' in category_name:
        return 'GIN'
    elif 'VODKA' in category_name:
        return 'VODKA'
    elif 'RUM' in category_name:
        return 'RUM'
    elif 'PUERTO RICO & VIRGIN ISLANDS RUM' in category_name:
        return 'RUM'
    elif 'BRANDY' in category_name or 'COGNAC' in category_name or 'BRANDIES' in category_name:
        return 'BRANDY'
    elif 'TEQUILA' in category_name:
        return 'TEQUILA'
    elif 'SCHNAPPS' in category_name:
        return 'SCHNAPPS'
    elif 'LIQUEUR' in category_name or 'AMARETTO' in category_name or 'CREME' in category_name or 'ANISETTE' in category_name:
        return 'LIQUEUR'
    elif 'CORDIAL' in category_name: 
        return 'CORDIAL'
    elif 'SPIRITS' in category_name or 'AMERICAN ALCOHOL' in category_name or 'DISTILLED' in category_name or 'MEZCAL' in category_name or 'DISTILLERIES' in category_name:
        return 'SPIRITS'
    elif 'COCKTAILS' in category_name or 'TRIPLE SEC' in category_name:
        return 'COCKTAILS'
    elif 'PACKAGES' in category_name or 'HOLIDAY VAP' in category_name or 'SPECIAL' in category_name:
        return 'SPECIALTY ITEM/PACKAGE'
    elif 'BEER' in category_name or 'ALE' in category_name or 'LAGER' in category_name or 'CIDER' in category_name:
        return 'BEER'
    elif 'DELISTED' in category_name:
        return 'DELISTED'
    else:
        return 'OTHER' 

# Example of applying the function
df['Main Category'] = df['Category Name'].apply(map_category)

### Clean Item Description

In [ ]:
import re

def clean_text(text):
    text = re.sub(r"'s|\.", "", text)  # Remove 's and .
    text = re.sub(r"[\\\-''\"]", "", text)  # Remove \, -, '', and "
    return text.strip()  

# Apply the cleaning function to the 'Item Description' column
df['Item Description'] = df['Item Description'].apply(clean_text)

In [ ]:
df.loc[df['Category Name'] == 'PUERTO RICO & VIRGIN ISLANDS RUM', 'Main Category'] = 'RUM'
df['Item Description'] = df['Item Description'].str.replace('SOOH', '', regex=False)
df['Item Description'] = df['Item Description'].str.replace(' FLAVORED VODKA', '', regex=False)
df['Item Description'] = df['Item Description'].str.replace(' VODKA', '', regex=False)
df['Item Description'] = df['Item Description'].str.replace(' BURNETTS', 'BURNETTS', regex=False)
df['Item Description'] = df['Item Description'].str.replace('  ', '', regex=False)

### Rename category

In [ ]:
category_cleaning = {
    # WHISKY
    'STRAIGHT BOURBON WHISKIES': 'AMERICAN WHISKIES',
    'TENNESSEE WHISKIES': 'AMERICAN WHISKIES',
    'BOTTLED IN BOND BOURBON': 'AMERICAN WHISKIES',
    'WHISKEY LIQUEUR': 'WHISKEY LIQUEURS',
    'STRAIGHT RYE WHISKIES': 'RYE WHISKIES',
    'SINGLE MALT SCOTCH': 'SCOTCH WHISKIES',
    'ROCK & RYE': 'WHISKEY LIQUEURS',
    # LIQUEURS
    'CREME DE ALMOND': 'CREAM LIQUEURS',
    'WHITE CREME DE CACAO': 'CREAM LIQUEURS',
    'GREEN CREME DE MENTHE': 'CREAM LIQUEURS',
    'DARK CREME DE CACAO': 'CREAM LIQUEURS',
    'WHITE CREME DE MENTHE': 'CREAM LIQUEURS',
    'IMPORTED AMARETTO': 'AMARETTO',
    'AMERICAN AMARETTO': 'AMARETTO',
    # SPIRITS
    'AMERICAN ALCOHOL': 'AMERICAN SPIRITS',
    'AMERICAN DISTILLED SPIRITS SPECIALTY': 'AMERICAN SPIRITS',
    'IOWA DISTILLERIES': 'IOWA SPIRITS', 
    'IMPORTED DISTILLED SPIRITS SPECIALTY': 'IMPORTED SPIRITS',
    # BRANDY
    'AMERICAN GRAPE BRANDIES': 'AMERICAN BRANDIES',
    # COCKTAILS
    'COCKTAILS /RTD':'COCKTAILS/RTD'
}

# Apply the cleaning
df['Category Name'] = df['Category Name'].replace(category_cleaning)
df['Category Name'] = df['Category Name'].str.replace('WHISKIES', 'WHISKY', regex=False)
df['Category Name'] = df['Category Name'].str.replace('BRANDIES', 'BRANDY', regex=False)

In [ ]:
replacements = {
    '100 PROOF VODKA': 'VODKA 100 PROOF',
    'DOMESTIC VODKAS': 'AMERICAN VODKAS',
    'IMPORTED FLAVORED VODKAS': 'IMPORTED FLAVORED VODKA',
    'IMPORTED VODKA - CHERRY': 'IMPORTED FLAVORED VODKA',
    'VODKAS': 'VODKA'
}

# Replace using the dictionary
df['Category Name'] = df['Category Name'].replace(replacements, regex=True)

In [ ]:
df.loc[df['Category Name'] == 'FLAVORED VODKA', 'Category Name'] = 'AMERICAN FLAVORED VODKA'

### Fill null values with mean and 'UNKNOWN' and '-1'


In [ ]:
# Fill County to UNKNOWN
df['County'].fillna('UNKNOWN', inplace=True)

# Fill Vendor Name with 'UNKNOWN'
df['Vendor Name'].fillna('UNKNOWN', inplace=True)

df['Vendor Number'].fillna('-1', inplace=True)

In [ ]:
# count all the null value into percentage

null_percentage = df.isnull().sum() / len(df) * 100
null_percentage

In [ ]:
# drop null value in state bottle cost, state bottle retail, sale dollars

df.dropna(subset=['State Bottle Cost', 'State Bottle Retail', 'Revenue'], inplace=True)

### Create New Dataframe

In [ ]:
# create new dataframe with only the columns we need

new_df = df[['Date', 'Store Number', 'County', 'Lat', 'Lon', 
         'Main Category', 'Category Name', 'Item Description', 'Vendor Name',  
         'Bottle Volume (ml)', 'State Bottle Cost', 'State Bottle Retail', 
         'Bottles Sold', 'Revenue', 'Volume Sold (Liters)']]

### Create Brand Name for Vodka

In [ ]:
# create a new df where the Main Category == VODKA

vodka_df = new_df[new_df["Main Category"] == 'VODKA']

In [ ]:
# fix name "SMINOFF SOURCED PINEAPPLE MINI" to "SMIRNOFF SOURCED PINEAPPLE MINI"

vodka_df.loc[vodka_df['Item Description'] == 'SMINOFF SOURCED PINEAPPLE MINI', 'Item Description'] = 'SMIRNOFF SOURCED PINEAPPLE MINI'


In [ ]:
brands = ["BILLSON","VODKA O", "MONOPOLOWA","KRU" , "PENTHOUSE SPIRITS", "MENAGE A TROIS", "HANGAR ONE", "OPULENT","BELAYA RUS", "KRYSHTAL ETALON", "MINSK KRISTALL", "PLATINKA", "FUZZYS", "A+", "JEWEL OF RUSSIA", "TOM OF FINLAND",
          "CRYSTAL HEAD", "ICEBERG", "PEARL", "POLAR ICE", "SCHRAMM ORGANIC", "DANZKA", "FRIS", "BLUE DIAMOND", "GORDON", "KIHNU MARK", "VIRU VALGE", "FINLANDIA", "KOSKENKORVA", "KORS VODKA", "STROM", "CIROC", "DRAGON BLEU",
          "ERISTOFF", "GREY GOOSE", "JEAN MARC XO", "MARIETTE", "KLEINER FEIGLING", "MAGIC SPIRITS", "RACHMANINOFF", "WODKA GORBATSCHOW", "ZARANOFF", "ROYAL DRAGON", "REYKA", "ROMANOV", "WHITE MISCHIEF", "L CHAIM", "KEGLEVICH",
          "VKA VODKA", "ROBERTO CAVALLI", "KISSUI", "CLIQUE", "LATVIJAS BALZAMS", "STUMBRAS", "OSO NEGRO", "VILLA LOBOS", "CHINGGIS", "BOLS", "BONG SPIRIT", "HOOGHOUDT", "KETEL ONE", "URSUS", "VAN GOGH", "VOX", "42 BELOW", "KUHL",
          "VIKINGFJORD", "MURREE VODKA", "BOLSKAYA", "WESTERN SON", "ABSOLWENT", "BELVEDERE", "BIALA DAMA", "CHOPIN", "CK VODKA", "CRACOVIA", "KRUPNIK", "LUKSUSOWA", "LANCUT", "POLONAISE", "SIWUCHA", "SOBIESKI", "SOPLICA", "STARKA", 
          "ULTIMAT", "WISENT", "WYBOROWA", "GOLD SYMPHONY", "HRENOVUHA", "KAUFFMAN", "KIZLYARKA", "KUBANSKAYA", "MOSKOVSKAYA", "NARODNAYA", "PUTINKA", "PYATIZVYOZDNAYA", "RODNIK", "RUSKOVA", "RUSSIAN STANDARD", "SHUSTOV", "STOLICHNAYA", 
          "STOLNAYA", "YOURI DOLGORUKI", "DOUBLE CROSS", "V44", "ABSOLUT", "EXPLORER", "KARLSSON", "LEVEL", "PINKY VODKA", "PURITY VODKA", "RENAT VODKA", "SVEDKA", "XELLENT SWISS", "KHLIBNYI DAR", "KHORTYTSIA", "NEMIROFF", "ADNAMS", 
          "BLACK COW", "CHASE", "GLEN", "ISENSUA", "THREE OLIVES", "VLADIVAR", "ANESTASIA VODKA", "BOWMAN", "BOYD & BLAIR", "CIRRUS", "DUBRA", "FELENE", "FIREFLY", "FLEISCHMANN", "GRAND TETON", "HANGAR 1", "LEOPOLD BROS.", "LOTUS", 
          "NEW AMSTERDAM", "NIKOLAI", "ORLOFF", "PINNACLE", "POPOV", "LA COLLECTION", "RAIN", "BURNETTS", "SAVVY VODKA", "SKYY", "SKINNYGIRL", "SQUARE ONE ORGANIC", "SAZERAC COMPANY", "TITO", "UV", "VAD", "LITTLE BLACK DRESS", 
          "VODKA 14", "SMIRNOFF", "WYCH DR DISTILLING", "HAKU", "360", "DEEP EDDY", "ODDKA", "KINKY", "JINRO","INGENIOZ", "EFFEN", "SEAGRAMS", "JEREMIAH", "BARTON", "TRULY", "PRAIRIE", "SMITHWORKS", "AMERICAN HARVEST", "OUTERSPACE",
          "BAKON", "STOLI", "SODA JERK", "HAWKEYE", "PRIDE OF THE WAPSI","HAMMER SICKLE","MR BOSTON","DEHNER DISTILLERY","SHADY FRUIT","HELLFYRE","MULLETS","PLUME & PETAL","GRIFFON","WISCONSIN CLUB","MEDEA", "RIVER BARON", "44 NORTH", 
          "MCCORMICK", "FIVE OCLOCK", "PLATINUM 7X", "CALIBER", "SWELL", "KORSKI", "PHILLIPS", "ROW", "TANQUERAY STERLING","SKOL", "BLUE OX", "CALIBER CITRUS", "RIVER PILOT", "FOUNDRY", "GOTCHA", "DUECES WILD", "THE ORIGINAL", "OCEAN ORGANIC", 
          "NATURAL LIGHT", "SWEET CAROLINA","PARAMOUNT", "CLEARHEART", "STATE", "OKOBOJI", "QUADRO RUSSIAN", "BAGGER", "HEROES", "BROKEN SHED", "NORTH FORTY", "FROST TRAIL", "REVELTON HONEY", "GRAYS PEAK", "WHEATLEY", "FIGENZA", "REVELTON","COLORADO HIGH", 
          "TRUST ME", "WHITE CLAW", "FIRESTARTER", "CENTURY FARMS", "SWARM", "WAVING PLAINS", "PUCKER", "ET 51", "FROZEN GHOST", "PLATINUM 10X", "MEILI", "KIRKLAND SIGNATURE", "RED FOX", "LORD PREMIUM","NORTH 40", "CHAMBORD", "BLUE ICE", "WILD ROOTS", 
          "IOWA LEGENDARY", "AMERICAN ANTHEM", "BLAUM BROS", "RELSKA", "WILD ROOTS PEAR", "BEDLAM", "BOMBORA", "WILD ROOTS PEACH", "JUST ANOTHER BLACK", "GILBEYS", "KIRKLAND", "SALUTE AMERICAN", "SARGES SPIRITS", "VOLI LYTE", "BISON GRASS", 
          "RIEGERS PREMIUM", "COLD ZERO", "CCD", "JUST ANOTHER", "QUADRO", "KING SOLLY", "FIRE IN THE HOLE", "ZYR VODKA", "JUST ANOTHER VODKA", "MEMBERS MARK", "ET51", "DOOR COUNTY", "WOODY CREEK DISTILLERS", "VEKTOR", "CHICKS", 
          "WOLFSCHMIDT", "CONCIERE VODKA", "MEMBERS MARK VODKA", "JJ OCONNELL", "DRAKES ORGANIC", "CM KNEE HIGH DISTILLING VODKA", "FRESH HERB VODKA", "PORTLAND POTATO","ROCKN", "HAVE SEXTRO", "SOLDIER VALLEY", "FRANKLY ORGANIC",
          "MUTINY ISLAND", "DEVILS SPRING", "KHORTYTSA", "YNOT", "COORANBONG", "DUFFYS RUN", "COPPER STILL", "ST GEORGE", "BUTCHERS BLOCK", "GRAY DUCK"]


In [ ]:
for item in brands:
    if brands.count(item) > 1:
        print(item)
def find_brand(description):
    for brand in brands:
        if brand in description:
            return brand
    return np.nan

vodka_df['Brand'] = vodka_df['Item Description'].apply(find_brand)

In [ ]:
# if the name is HAIR OF THE DAWG VODKA, the brand is TITO

vodka_df.loc[vodka_df['Item Description'] == 'HAIR OF THE DAWG VODKA', 'Brand'] = 'TITO'

In [ ]:
vodka_df['Brand'].nunique()
pd.set_option('display.max_rows', None)

vodka_null = vodka_df[(vodka_df['Main Category'] == 'VODKA') & (vodka_df['Brand'].isnull())]

# Count occurrences of each unique 'Item Description' in these rows
vodka_null_counts = vodka_null['Item Description'].value_counts()

# Print the counts
print(vodka_null_counts)

In [ ]:
vodka_df['Brand'] = vodka_df['Brand'].fillna('OTHERS')

### Save to new csv

In [ ]:
vodka_df.to_csv('vodka_data.csv', index=False)

In [ ]:
# save new df to csv

new_df.to_csv('cleaned_data.csv', index=False)